In [57]:
from secret_keys import openai_key
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
os.environ['OPENAI_API_KEY'] = openai_key

In [ ]:
from langchain.llms import OpenAI

# temperature param:- ratio of creative model
                    # 1 is risky model but creative. Mostly used 0.6 or 0.9
                    # 0 no risk
llm = OpenAI(temperature=0.6)
name = llm("I want to open restaturant for Indian food. Suggest a fancy name for this.")
print(name)

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables= ['cuisine'],
    template= "I want to open restaturant for {cuisine} food. Suggest a fancy name for this."
    )
prompt_template_name.format(cuisine="Indian")

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.6)

# Chain 1
prompt_template_name = PromptTemplate(
    input_variables= ['cuisine'],
    template= "I want to open restaturant for {cuisine} food. Suggest a fancy name for this."
    )
name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

# Chain 2
prompt_template_name = PromptTemplate(
    input_variables= ['resto_name'],
    template= """Suggest some menu items for {resto_name}. Return it as a coma separated list"""
    )
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_name)

# RUN Chains
#simple seq chain example

from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
response = chain.run("Indian")
print(response)
# remeber the response will of last chain only in this case response will be of food_items_chain.

In [ ]:
# Here we have the example of the seq chain which will give us the output of all the required chains

# Chain 1
prompt_template_name = PromptTemplate(
    input_variables= ['cuisine'],
    template= "I want to open restaturant for {cuisine} food. Suggest a fancy name for this."
    )
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="resto_name")

# Chain 2
prompt_template_name = PromptTemplate(
    input_variables= ['resto_name'],
    template= """Suggest some menu items for {resto_name}. """
    )
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="menu_items")

from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['resto_name', 'menu_items']
)

chain(
    {
        'cuisine': 'Indian'
    }
)

# AGENTS Creation

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.6)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent.run("When was Elon Musk was born? What was his age in 2023?")

# SerpAPI

In [54]:
from secret_keys import serpapi_key
from langchain.agents import Tool
from langchain_community.utilities import SerpAPIWrapper

os.environ['SERPAPI_API_KEY'] = serpapi_key

#### New Way

In [ ]:
# New Way 
llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

print(agent.run("What was the GDP of US in 2022 plus 5?"))

#### OLD WAY TO DO IT

In [ ]:
## OLD WAY TO DO IT

# llm = OpenAI(temperature=0)

# # the tool we'll give the agent access to. Note that the 'llm-math' tool uses an LLM, 
# tools = load_tools(["serpapi", "llm-math"], llm=llm)
# #search = SerpAPIWrapper()  # uses os.environ["SERPAPI_API_KEY"]
# # tools = [
# #     Tool(
# #         name="Search",
# #         func=search.run,
# #         description="useful for answering questions about current events"
# #     )
# # ]

# # Finally, let's initialize an agent with the tools, the langauage model, and the type 
# agent = initialize_agent(tools, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# agent.run("What was the GDP of US in 2022 plus 5?")

# MEMORY

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

# expected output: Taco Fiesta

In [ ]:
name = chain.run("Indian")
print(name)

# expected output: Maharaja's Palace Cuisine

In [ ]:
chain.memory
# conversation with alignment
print(chain.memory.buffer)

In [ ]:
# ConversationBufferMemory this saves every ques and answer (combined 1 conversational exchange). This will send the all past history to OpenAI and OpenAI charges you per token.
# so reduce the buffer size 
# example save only last 5 exchange using Conversation exchange 

### Conversation Chain

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
convo.prompt

print(convo.prompt.template)

In [ ]:
convo.run("who won first cricket world cup?")

#expected output: West Indies

In [ ]:
convo.run("5+5")

# Expected output: 10 

In [ ]:
convo.run("who was the captain of the winning team?")

#expected output: captain name

In [ ]:
convo.memory

#expected output: All the past history

In [ ]:
from langchain.memory import ConversationBufferMemory
# k=1 sets means remeber only last 1 convertional exchange 
memory = ConversationBufferMemory(k=1)
convo = ConversationChain(llm=OpenAI(temperature=0.7), memory=memory)

convo.run("who won first cricket world cup?")

#expected output: West Indies

In [ ]:
convo.run("5+5")

# Expected output: 10 

In [ ]:
convo.run("who was the captain of the winning team?")

# expected output: I'm sorry, I don't know

### As the conversationChain is deprecated following code is using RunnableWithMessageHistory


In [ ]:
# from langchain_openai import OpenAI 
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory

# Initialize the model
llm = OpenAI(temperature=0.7)

# Create the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

# Connect prompt and model
chain = prompt | llm

# In-memory session history store
session_histories = {}

def get_history(session_id: str):
    if session_id not in session_histories:
        session_histories[session_id] = InMemoryChatMessageHistory()
    return session_histories[session_id]

# Wrap chain with message history handling
runnable = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key="input",
    history_messages_key="history",
)

# Run it
response = runnable.invoke(
    {"input": "Who is Elon Musk?"},
    config={"configurable": {"session_id": "test-session"}}
)

print(response.content)
